# Sandobox for developing bounding box detectors for orthomosaics
#### input: an orthomosaic and a YOLOv5 model
#### output: multi-polygon (bounding boxes) shapefile with bounding box class and probability

In [1]:
import os, glob, shutil
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")      
import subprocess
import sys

#from ocifs import OCIFileSystem
#import fsspec
from datetime import datetime, timedelta
from osgeo import gdal, ogr, osr
import pandas as pd
import geopandas as gpd
from pathlib import Path
#import logging
import os
import sys
import tkinter as tk
import tkinter.filedialog as fd

# for inference part
import torch
import cv2
#from cv2 import cv2
import PIL.Image as Image
from io import StringIO
from io import BytesIO

from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import rasterio as rio
import numpy as np

# for parsing part
from utils.tools import yolo2xy


gdal_utils_path=os.getcwd()+"/utils/"
modelZoo_path=os.getcwd()+"/model_zoo/"
yolov5_repo_path=os.getcwd()+"/yolov5"


print("Start timestamp in UTC: {}".format(str(datetime.utcnow())))

Start timestamp in UTC: 2022-12-07 14:18:09.250990


In [2]:
def directory_mode():
    root = tk.Tk()
    orthos_to_process = []
    directory = fd.askdirectory(parent=root, title="Choose directory")
    input_orthos_to_process = glob.glob(directory + "/**/*.tif", recursive=True)
    for i in input_orthos_to_process:
        if "FSCT_output" not in i:
            orthos_to_process.append(i)
    root.destroy()
    return orthos_to_process


In [3]:
# DEFINE MAIN PARAMETERS
tile_size_m=32
buffer_size_m=2
format_tiles="GTiff"
GPU = False

# define inference parameters
img_size= 640
conf_threshold=0.4 # confidence threshold to remove bounding boxes from inference 

intile=1 # tile inner buffer in meters to remove edge artifacts                                       
iou_thresh=0.75 # used to clea-up duplicate boxes. It is the maximum threshold above which two intersecting bounding boxes are considered as the same one and thus the one with largest probability is selected
model_name="treeSpecies_detector_s_im640_batch20_best.pt"

In [4]:
# Define classes
d = {'class_name': ["bo","spruce","pine","deciduous"], 'class_code': [0,1,2,3]}
classes = pd.DataFrame(data=d)
classes

,class_name,class_code
0,bo,0
1,spruce,1
2,pine,2
3,deciduous,3


In [5]:
# OCI-related

#logging.getLogger("ocifs").setLevel(logging.ERROR)
#logging.getLogger("yolov5").setLevel(logging.ERROR)
 
#os.system("conda-unpack")

#def copyFileRemotely(fileFrom,fileTo):
#    fs = OCIFileSystem()
#    input = open(fileFrom,"rb")
#    output = fs.open(fileTo,"wb")
#    output.write(input.read())
#    output.close()
#    input.close()
    
#def copyFileLocally(fileFrom,fileTo):
#    fs = OCIFileSystem()
#    input = fs.open(fileFrom)
#    output = open(fileTo,"wb")
#    output.write(input.read())
#    output.close()
#    input.close()

In [6]:
## parse input parameters
#for item1 in sys.argv[1:]:
#    item = item1.split('=',2)
#    os.environ[item[0]]=item[1]

#input_location = os.environ.get("OBJ_INPUT_LOCATION")
#output_location = os.environ.get("OBJ_OUTPUT_LOCATION")

#if input_location is not None:
#    print("Input location: "+ input_location)
#else:
#    print("Missing input location (OBJ_INPUT_LOCATION)")
#    exit
    
#if output_location is not None:
#    print("Output location: "+ output_location)
#else:
#    print("Missing ouput location (OBJ_OUTPUT_LOCATION)")
#    exit


In [7]:
# if executed as a job, the conda pack is in /condapack folder and the artifact in decompressed_artifact
#   decompressed_artifact/nibio/inference.py
#   /home/datascience/condapack/yolov5/lib/python3.8/site-packages/torch/hub.py

#JOB_RUN_OCID_KEY = "JOB_RUN_OCID"
#job_run_ocid = os.environ.get(JOB_RUN_OCID_KEY, "UNDEFINED")

# to work in jobs
#dirname = os.path.dirname(os.path.abspath(__file__))
#print("dirname= " + dirname)

### create a temporary directory with a tiles_dir subdirectoruy
#local_temp="/home/datascience/tmp/"
#if not os.path.exists(local_temp):
#    os.makedirs(local_temp)    
    
#tiles_dir_temp=local_temp+"tiles_dir/"
#if not os.path.exists(tiles_dir_temp):
#    os.makedirs(tiles_dir_temp)    

In [8]:
# get list of orthomosaics to process
orthos_to_process = directory_mode()
orthos_to_process

['C:/Users/stpu/snowBreakYOLO/predictLeafOFF\\Avskakalia_1st_ortho.tif',
 'C:/Users/stpu/snowBreakYOLO/predictLeafOFF\\Brattaas_1st_ortho.tif',
 'C:/Users/stpu/snowBreakYOLO/predictLeafOFF\\dahl_1st_ortho.tif',
 'C:/Users/stpu/snowBreakYOLO/predictLeafOFF\\Ersgaard_1st_ortho.tif',
 'C:/Users/stpu/snowBreakYOLO/predictLeafOFF\\huuse_1st_ortho.tif',
 'C:/Users/stpu/snowBreakYOLO/predictLeafOFF\\Jonsonhaugen_1st_ortho.tif',
 'C:/Users/stpu/snowBreakYOLO/predictLeafOFF\\Sustad_1st_ortho.tif']

In [9]:
for ortho_path in orthos_to_process:
    # 1 - SETUP DIRS
    print("Started with: "+ ortho_path)
    ortho_name=Path(ortho_path).stem # ortho name      
    temp= os.path.dirname(ortho_path)
    if not os.path.exists(temp):
        os.makedirs(temp)    
    tiles_temp=temp+"/tiles_dir_"+ortho_name+"/"
    if not os.path.exists(tiles_temp):
        os.makedirs(tiles_temp)    
    temp_predict=tiles_temp+"predict"
    if not os.path.exists(temp_predict):
        os.makedirs(temp_predict)
    temp_dir_predict_out=temp_predict+"/out"
    temp_dir_labels=temp_dir_predict_out+"/labels"
    
    
    
    # 2 - GET RASTER METADATA
    print("Loading orthomosaic and extracting some metadata")
    ## Get pixel resolution (in meters) and tile size in pixels
    src_ds = gdal.Open(ortho_path)
    _, xres, _, _, _, yres  = src_ds.GetGeoTransform() # get pixel size in meters
    tile_size_px= round(tile_size_m/abs(xres)) # calculate the tile size in pixels
    ## Get EPSG code
    proj = osr.SpatialReference(wkt=src_ds.GetProjection())
    EPSG_code= proj.GetAttrValue('AUTHORITY',1)
    
    
    
    # 3 - GENERATE ORTHOMOSAIC BOUNDARY SHAPEFILE
    print("Generating orthomosaic boundary file")
    ## Define name for boundary shapefile
    shape_path=temp+"/"+ortho_name+"_boundary.shp"
    ## Run gdal_polygonize.py to get boundaries from alpha band (band 4)
    #%run /home/datascience/cnn_wheel_ruts/gdal_polygonize.py $ortho_path -b 4 $shape_path
    command_polygonize ="python "+ gdal_utils_path+"gdal_polygonize.py "+ ortho_path + " -b 4 " + shape_path
    print(os.popen(command_polygonize).read())
    ## Select polygon that has DN equal to 255, indicating the area where drone data is available for
    polys = gpd.read_file(shape_path)
    polys[polys['DN']==255]#.to_file(shape_path)
    polys.to_file(shape_path)
    
    
    
    # 4 - TILING THE ORTHOMOSAIC
    print("Tiling the orthomosaic")
    ## Define buffer size and calculate the size of tiles excluding buffer
    tile_size_px= round(tile_size_m/abs(xres))
    buffer_size_px= round(buffer_size_m/abs(xres))
    tileIndex_name=ortho_name+"_tile_index" # define name for output tile index shapefile
    ## Run gdal_retile.py (can take some minutes) 
    if format_tiles=="PNG":
        command_retile = "python "+ gdal_utils_path +"gdal_retile.py -targetDir " + tiles_temp + " " + ortho_path+ " -overlap " + str(buffer_size_px) + " -ps "+str(tile_size_px) + " " + str(tile_size_px) + " -of PNG -co WORLDFILE=YES -tileIndex "+ tileIndex_name + " -tileIndexField ID"
    if format_tiles=="GTiff":
        command_retile = "python "+ gdal_utils_path+ "gdal_retile.py -targetDir " + tiles_temp + " " + ortho_path+ " -overlap " + str(buffer_size_px) + " -ps "+str(tile_size_px) + " " + str(tile_size_px) + " -of GTiff -tileIndex "+ tileIndex_name + " -tileIndexField ID"
    print(os.popen(command_retile).read())
    
    
    # 5 - KEEP ONLY TILES WITHIN THE ORTHOMOSAIC BOUNDARY
    print("Removing boundary tiles")
    ## Load boundary
    boundary = gpd.read_file(shape_path) #  read in the shapefile using geopandas
    boundary = boundary.geometry.unary_union #union of all geometries in the GeoSeries
    ## Load tiles shapefile
    tiles = gpd.read_file(tiles_temp+ "/"+ortho_name+"_tile_index.shp")
    ## Select all tiles that are not within the boundary polygon
    tiles_out = tiles[~tiles.geometry.within(boundary)]
    ## Create a series for each file format with all names of files to be removed
    names_tiles_out = [os.path.splitext(x)[0] for x in tiles_out['ID']] # get names without extension

    if format_tiles=="PNG":
        pngs_delete=[tiles_dir+ "/"+sub + '.png' for sub in names_tiles_out] # add .png extension
        xml_delete=[tiles_dir+ "/" +sub + '.png.tmp.aux.xml' for sub in names_tiles_out] # ...
        wld_delete=[tiles_dir+ "/"+sub + '.png.wld' for sub in names_tiles_out] #...
        ## Delete files
        for f in pngs_delete: # delete png files
            if os.path.exists(f):
                os.remove(f)
        for f in xml_delete:  # delete xmls files
            if os.path.exists(f):
                os.remove(f)
        for f in wld_delete:  # delete world files
            if os.path.exists(f):
                os.remove(f)

    if format_tiles=="GTiff":
        

        gtiffs_delete=[tiles_temp+ "/"+sub + '.tif' for sub in names_tiles_out] # add .png extension
        for f in gtiffs_delete: # delete png files
            if os.path.exists(f):
                os.remove(f)
        print("    Removing " +str(len(gtiffs_delete))+" tiles")
        print("    Remaining " +str(len(tiles)-len(gtiffs_delete))+" tiles")       
    print("Finished with ortho tiling part ..................................................................................")

    
    
    # 6 - YOLO INFERENCE
    print("Starting YOLO inference (this might take a while without GPU)")
    ## define model weights file
    model_weights =modelZoo_path+"/"+model_name
    print("Predicting using this model: " + model_weights)   
    ## run inference
    if GPU == True:
        command_predict = "python "+ yolov5_repo_path+"/detect.py --source " + tiles_temp + " --weights " + model_weights + " --img " + str(img_size) + " --name " + temp_dir_predict_out + " --save-txt --save-conf --nosave --conf-thres " + str(conf_threshold) + " --device=0" + " --agnostic"
        #%run /home/datascience/conda/yolov5/yolov5/detect.py --source $temp_dir_predict --weights $model_weights --img 640  --name $temp_dir_predict --save-txt --save-conf --nosave --conf-thres=0.4 --device=0
    else:
        command_predict = "python "+ yolov5_repo_path+ "/detect.py --source " + tiles_temp + " --weights " + model_weights + " --img " + str(img_size) + " --name " + temp_dir_predict_out + " --save-txt --save-conf --nosave --conf-thres " + str(conf_threshold) + " --agnostic"
        # %run /home/datascience/conda/yolov5/yolov5/detect.py --source $temp_dir_predict --weights $model_weights --img 640  --name $temp_dir_predict --save-txt --save-conf --nosave --conf-thres=0.4

    print(os.popen(command_predict).read())
    print("Finished YOLO inference ..........................................................................................")
    
    
    
    # 7 - PARSING FROM IMAGE SPACE TO MAP SPACE
    print("Starting to parse the predicted bounding boxes from image to map space")
    gtiffs = glob.glob(tiles_temp + "/*.tif")
    labels = glob.glob(temp_dir_labels + "/*.txt")

    tile_index_shape = glob.glob(tiles_temp + "/" + "*tile_index*") 
    for l in tile_index_shape:
        l

    tile_index_temp = tiles_temp+ "/" + os.path.splitext(os.path.basename(l))[0] + ".shp"
    tile_index = gpd.read_file(tile_index_temp) # read tile index shapefile

    ## Get pixel resolution (in meters) and tile size in pixels
    src_ds = gdal.Open(gtiffs[0]) # get raster datasource
    _, xres, _, _, _, yres  = src_ds.GetGeoTransform() # get pixel size in meters
    tile_size_m=round(src_ds.RasterXSize*xres)
    tile_size_px= round(tile_size_m/abs(xres)) # calculate the tile size in pixels
    ## Get EPSG code
    proj = osr.SpatialReference(wkt=src_ds.GetProjection())
    EPSG_code= proj.GetAttrValue('AUTHORITY',1)
    
    # iterate through each txt file and trhough each row (bounding box) within a txt file
    all_bboxes = None
    iter_all=0
    for lab in range(len(labels)):
        print(str(round(lab/len(labels)*100))+" % done!")
        # Define one label file and select the corresponding geotiff image
        label_file=labels[lab]
        label_file_name=Path(label_file).stem # ortho name
        for p in gtiffs:
            if Path(p).stem ==label_file_name:
                gtiff_file=p

        #print("debug label file = " + label_file)
        #print("debug tif file = " + gtiff_file)

        # determing image witdth and height
        r = gdal.Open(gtiff_file)
        img_width=r.RasterXSize
        img_height=r.RasterYSize

        # Convert from yolo coordinates to x1, y1, x2, y2,
        # WARNING : I had to modify yolo2xyx with ocifs + other things
        coords = yolo2xy(label_file, img_width, img_height) # class, x1, y1, x2, y2, probability 

        # Convert from image to geographical coordinates
        ## select tile polygon (from tile index shapefile) that corresponds to the label_file_name
        # the other files are required by the gpd readfile


        # tile_index is <class 'geopandas.geodataframe.GeoDataFrame'>
        one_tile=tile_index[tile_index['ID']==label_file_name+".tif"] # Select tile in tile_index that has ID equal to label_file_name

        ## get tile bounding box geographical coordinates (UTM)
        one_tile_XminUTM=one_tile.total_bounds[0]
        one_tile_YminUTM=one_tile.total_bounds[1]
        one_tile_XmaxUTM=one_tile.total_bounds[2]
        one_tile_YmaxUTM=one_tile.total_bounds[3]

        ## take inner buffer equal to the buffer_size_m 
        one_tile_innerB= one_tile
        one_tile_innerB['geometry'] = one_tile_innerB.geometry.buffer(-intile)

        ## get inner tile bounding boxes
        one_tile_inner_XminUTM=one_tile_innerB.total_bounds[0]
        one_tile_inner_YminUTM=one_tile_innerB.total_bounds[1]
        one_tile_inner_XmaxUTM=one_tile_innerB.total_bounds[2]
        one_tile_inner_YmaxUTM=one_tile_innerB.total_bounds[3]

        # Now iterate through each bounding box and assign UTM coordinates and create a shapefile
        bboxes_tile = None
        for i in coords:
            # print("inside coords")
            # Convert bounding box coordinates from image to geographical coords
            X1_UTM=(i[1]*xres)+one_tile_XminUTM
            Y1_UTM=(i[2]*yres)+one_tile_YminUTM+tile_size_m
            X2_UTM=(i[3]*xres)+one_tile_XminUTM
            Y2_UTM=(i[4]*yres)+one_tile_YminUTM+tile_size_m

            # skip bounding box if its centroid is NOT within the inner tile (removing the overlap)
            X_UTM= (X1_UTM+X2_UTM)/2
            Y_UTM= (Y1_UTM+Y2_UTM)/2
            if X_UTM<one_tile_inner_XminUTM or X_UTM>one_tile_inner_XmaxUTM or Y_UTM<one_tile_inner_YminUTM or Y_UTM>one_tile_inner_YmaxUTM:
                #print("continue break")
                continue

            #print("over continue break")    

            # Create polygon shape from geographical coords
            lat_point_list = [Y1_UTM, Y1_UTM, Y2_UTM, Y2_UTM, Y1_UTM]
            lon_point_list = [X1_UTM, X2_UTM, X2_UTM, X1_UTM, X1_UTM]
            polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
            crs = {'init': 'epsg:'+EPSG_code}
            data= {'class': [i[0]], 'prob': [i[5]]}
            bbox = gpd.GeoDataFrame(data, crs=crs, geometry=[polygon_geom])

            if (bboxes_tile is None):
                bboxes_tile = bbox
            else:
                bboxes_tile = bboxes_tile.append(bbox)

        # cleanup boxes (removing overlapping ones)
        if bboxes_tile is not None:
            #clean_boxes = bboxes_tile #cleanUp_boudingBoxes(bboxes_tile, iou_thresh)
            if (all_bboxes is None):
                all_bboxes = bboxes_tile
            else :
                all_bboxes = all_bboxes.append(bboxes_tile)
                
                
    # convert class names
    all_bboxes.loc[all_bboxes['class']==0,'class_name']='bo'
    all_bboxes.loc[all_bboxes['class']==1,'class_name']='spruce'
    all_bboxes.loc[all_bboxes['class']==2,'class_name']='pine'
    all_bboxes.loc[all_bboxes['class']==3,'class_name']='deciduous'
    all_bboxes= all_bboxes.drop(['class'],axis=1)
    
    # export shapefile
    all_bboxes.to_file(temp + "/" + ortho_name + "_predictions.shp", driver='ESRI Shapefile') # turn this off if it's not needed
    
    # convert to geoJSON
    
    # convert to geoJSON
    print("converting geoJSON.............")
    input_shp = temp + "/" + ortho_name + "_predictions.shp"
    output_geoJson = temp + "/" + ortho_name + "_predictions.json"
    cmd = "ogr2ogr -f GeoJSON "  + output_geoJson +" " + input_shp
    print(cmd)
    subprocess.call(cmd , shell=True)       
    print("Finished file.........................................................................................................")
    print("......................................................................................................................")
    print("......................................................................................................................")
    print("......................................................................................................................")
    
    #shutil.rmtree(tiles_temp)


    





Started with: C:/Users/stpu/snowBreakYOLO/predictLeafOFF\Avskakalia_1st_ortho.tif
Loading orthomosaic and extracting some metadata
Generating orthomosaic boundary file
0...10...20...30...40...50...60...70...80...90...Creating output C:/Users/stpu/snowBreakYOLO/predictLeafOFF/Avskakalia_1st_ortho_boundary.shp of format ESRI Shapefile.
100 - done.

Tiling the orthomosaic
0...10...20...30...40...50...60...70...80...90...100 - done.

Removing boundary tiles
    Removing 24 tiles
    Remaining 130 tiles
Finished with ortho tiling part ..................................................................................
Starting YOLO inference (this might take a while without GPU)
Predicting using this model: C:\Users\stpu\snowBreakYOLO/model_zoo//treeSpecies_detector_s_im640_batch20_best.pt

Finished YOLO inference ..........................................................................................
Starting to parse the predicted bounding boxes from image to map space
0 % done!
1 % done!